### Google Image download with full-scroll

In [1]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
import os
import json
from urllib.request import *
import sys
import time

As you are using Selenium 3.8.0 you have to use GeckoDriver mandatorily. But again as you are using Firefox v46.0 you have to set the capability marionette to False through `DesiredCapabilities()` as follows :

```python
cap = DesiredCapabilities().FIREFOX
cap["marionette"] = True
```

**Current version**

- `geckodriver` : v0.24.0

- `FireFox` : 68.0.1

In [2]:
# adding path to geckodriver to the OS environment variable
os.environ["PATH"] += os.pathsep + os.getcwd()
download_path = "images/"

In [3]:
searchtext = "t shirt transparent background"
searchtext = searchtext.split(' ')
searchtext = '+'.join(searchtext)

# number_of_scrolls * 400 images will be opened in the browser
num_requested = 500
number_of_scrolls = num_requested / 400 + 1 

In [4]:
if not os.path.exists(download_path + searchtext.replace("+", "_")):
    os.makedirs(download_path + searchtext.replace("+", "_"))

In [5]:
url = "https://www.google.co.in/search?q="+searchtext+"&source=lnms&tbm=isch"
# driver = webdriver.Firefox()
# driver = webdriver.Firefox(capabilities=cap) # executable_path="C:\\path\\to\\geckodriver.exe"
driver = webdriver.Firefox(executable_path="./geckodriver")
driver.get(url)

In [6]:
headers = {}
headers['User-Agent'] = "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36"
extensions = {"jpg", "jpeg", "png"} #, "gif"
img_count = 0
downloaded_img_count = 0

In [7]:
for _ in range(int(number_of_scrolls)):
    for __ in range(10):
        # multiple scrolls needed to show all 400 images
        driver.execute_script("window.scrollBy(0, 1000000)")
        time.sleep(0.2)
    # to load next 400 images
    time.sleep(0.5)
    try:
        driver.find_element_by_xpath("//input[@value='결과 더보기']").click()
    except Exception as e:
        print ("Less images found: {}".format(e))
        print ("if it seems that scrolling is done, just pass to next step.")
        break

Less images found: Message: Element <input id="smb" class="ksb" type="button"> could not be scrolled into view

if it seems that scrolling is done, just pass to next step.


In [8]:
imges = driver.find_elements_by_xpath('//div[contains(@class,"rg_meta")]')
print ("Total images: {}\n".format(len(imges)))

Total images: 493



In [ ]:
for img in imges:
    img_count += 1
    img_url = json.loads(img.get_attribute('innerHTML'))["ou"]
    img_type = json.loads(img.get_attribute('innerHTML'))["ity"]
    print ("Downloading image {}:{}".format(img_count,img_url))
    try:
        if img_type not in extensions:
            img_type = "jpg"
        req = Request(img_url, headers=headers)
        raw_img = urlopen(req).read()
        f = open(download_path+searchtext.replace("+", "_")+"/"+str(downloaded_img_count)+"."+img_type, "wb")
        f.write(raw_img)
        f.close
        downloaded_img_count += 1
    except Exception as e:
        print ("Download failed: {}".format(e))
    finally:
        print
    if downloaded_img_count >= num_requested:
        break

print("download complete.")
print("Total downloaded: {}/{}".format(downloaded_img_count,img_count))
driver.quit()

---

In [10]:
print("download complete.")
print("Total downloaded: {}/{}".format(downloaded_img_count,img_count))
driver.quit()

download complete.
Total downloaded: 485/493
